## Densidad de materia oscura

### Informacion por utilizar

In [1]:
#importamos las herramientas necesarias
import pynbody
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy import optimize
import scipy.stats
from astropy import units as unit
import sys
from matplotlib.animation import FuncAnimation
import Funciones_Dark_Matter as fnd

In [2]:
#importamos los datos
data_nuevo = '/media/danna01/Disk/SNAPSHOTS/Galaxias_Nuevas'
data_viejo = '/media/danna01/Disk/SNAPSHOTS/Galaxias_Viejas'
#datadir = '/media/juan/Pesquisa/Simulations_Chris_2021'

A continuacion se lista la informacion importante para este notebook

 1) Masa Total de materia oscura
 
 2) Densidad de la materia oscura

In [3]:
# Estos son los diccionarios que definen los limites que seran utilizados para los plots de las galaxias para los
# calculos de densidad de materia oscura y la masa acumulada a diferentes radios para la amteria oscura tambien
dicti = {}
galaxies = ['G0', 'G1', 'G2', 'G3']
# Estos son los limites para graficar la masa acumulada y la densidad; sus valores corresponden a 10 veces el valor
# del scale - lenght de cada galaxia.
limites_materia_oscura = {'G0':(0,55), 'G1':(0,75), 'G2':(0,95), 'G3':(0,140)} #kpc
# Aca observamos los limites que se usaran para calcular el valor de 'h' que nos indicara que tan bueno es el ajuste
# limites --> De que dependen?

### Funciones Comparaciones

#### Relacion masa acumulada vs. radio

In [42]:
# Masa acumulada de materia oscura
def accumulated_mass(posnew,massnew,posold, massold,n,limits):
    # Para las simulaciones nuevas
    rnew = np.sqrt((posnew**2).sum(axis=1))
    # histograma acumulado
    histo_acum_new = scipy.stats.cumfreq(rnew, n, limits, weights=massnew)
    # hallamos los radios manualmente
    positionsnew = histo_acum_new[1]+ np.arange(1,n+1) * histo_acum_new[2]
    # Para las simulaciones viejas
    rold = np.sqrt((posold**2).sum(axis=1))
    histo_acum_old = scipy.stats.cumfreq(rold, n, limits, weights=massold)
    positionsold = histo_acum_old[1]+ np.arange(1,n+1) * histo_acum_old[2]
    
    return positionsnew, histo_acum_new[0], positionsold, histo_acum_old[0]

#### Relacion densidad vs. radio

In [43]:
# Funcion densidad
def density_dm(xnew,ynew,xold,yold):
    xnew,ynew, xold, yold = accumulated_mass(data_nuevo_ruta.dm['pos'],data_nuevo_ruta.dm['mass'],data_viejo_ruta.dm['pos'],data_viejo_ruta.dm['mass'],n=int(limites_materia_oscura[gal][1]*4),limits=limites_materia_oscura[gal])
    # Para las simulaciones nuevas
    # np.roll re acomoda los datos desplazandolos una posicion
    x2new = np.roll(xnew,1)
    x2new[0]=0
    # promedio del radio
    bins_middle_new = (xnew + x2new )/2
    
    x_cubico_new = xnew**3
    x_cubico_2_new = np.roll(x_cubico_new,1)
    x_cubico_2_new[0]=0
   
    y2new = np.roll(ynew,1)
    y2new[0]=0
  
    shell_mass_new = ynew - y2new
    
    # \rho = (4/3)*pi*R^3
    densnew = 3* shell_mass_new / ((4*np.pi)* (x_cubico_new - x_cubico_2_new) )
    
    # Para las simulaciones viejas
    x2old = np.roll(xold,1)
    x2old[0]=0
    bins_middle_old = (xold + x2old )/2
    
    x_cubico_old = xold**3
    x_cubico_2_old = np.roll(x_cubico_old,1)
    x_cubico_2_old[0]=0
    
    y2old = np.roll(yold,1)
    y2old[0]=0
   
    shell_mass_old = yold - y2old
    
    # \rho = (4/3)*pi*R^3
    densold = 3* shell_mass_old / ((4*np.pi)* (x_cubico_old - x_cubico_2_old) ) 
    
    return bins_middle_new, shell_mass_new, densnew, bins_middle_old, shell_mass_old, densold

#### Funciones para las graficas

In [44]:
# Plots masa acumulada.
def plot_mass(binsnew,massnew,binsold,massold,filename):
    fig = plt.figure(figsize=(10, 7))
    
    plt.plot(binsnew,massnew,'-k',label= 'New Galaxies')
    plt.plot(binsold,massold,'-r', label= 'Old Galaxies')
    
    plt.title('Masa acumulada de la Materia Oscura', fontsize= 18)
    plt.xlabel('Radio [Kpc]',fontsize=20)
    plt.ylabel('Masa Acumulada (Msol)')
    plt.legend()
    plt.grid()
    plt.savefig(filename)
    plt.clf()
    plt.close()

In [47]:
# Plots densidad.
def plot_density(binsnew,densnew,binsold, densold, filename):
    
    fig = plt.figure(figsize=(10, 7))
    
    plt.plot(binsnew, np.log10(densnew), '-k', label= 'New Galaxies')
    plt.plot(binsold, np.log10(densold), '-r', label= 'Old Galaxies')
   
    plt.title('Perfil de densidad de materia oscura', fontsize = 18)
    plt.xlabel('Radio [Kpc]', fontsize = 20)
    plt.ylabel(r'$\rho$ [$1x10^{10}$ Msol/$kpc^{2}$]', fontsize = 20)
    plt.legend()
    plt.grid()
    plt.savefig(filename)
    plt.clf()
    plt.close()

#### Generacion de graficas

In [55]:
# Generacion de plots.
minsnap = 100
maxsnap = 700
stepsnap = 200

# definimos la fuente de los datos, para esto usamos los diccionarios que definimos mas arriba
for gal in ['G0','G1']:
    for snap in np.arange(minsnap,maxsnap,stepsnap): 
        snap = str(snap).zfill(3)
        # Aca definimos la ruta en la cual estan almacenados los snapshots
        data_nuevo_ruta = pynbody.load(data_nuevo+'/'+gal+'/snapshot_'+snap+'.hdf5')
        data_viejo_ruta = pynbody.load(data_viejo+'/'+gal+'/snapshot_'+snap+'.hdf5')
        
        
        # Aca vamos a generar cada uno de los plots, para lo cual primero debemos definir
        # cada una de las funciones de masa acumulada y densidad.
        bins_new, mass_new, bins_old, mass_old = accumulated_mass(data_nuevo_ruta.dm['pos'],data_nuevo_ruta.dm['mass'],data_viejo_ruta.dm['pos'],data_viejo_ruta.dm['mass'],n=int(limites_materia_oscura[gal][1]*4),limits=limites_materia_oscura[gal])
    
        bins_middle_new, shell_mass_new, dens_new, bins_middle_old, shell_mass_old, dens_old  = density_dm(bins_new, mass_new, bins_old, mass_old)
      
        plot_mass(bins_new,mass_new,bins_old,mass_old,filename='Materia_oscura/Comparaciones/Materia_Oscura_Acumulada_'+gal+'_dm_'+snap+'.jpg')
        
        plot_density(bins_middle_new,dens_new,bins_middle_old,dens_old,filename='Materia_oscura/Comparaciones/Materia_Oscura_Densidad_'+gal+'_dm_'+snap+'.jpg')

/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No unit information found: using gadget-units.
  warnings.warn(
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No unit information found: using gadget-units.
  warnings.warn(
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No un

### Funciones Materia Oscura Galaxias Nuevas

#### Relacion masa acumulada vs. radio

In [4]:
# Masa acumulada de la materia oscura

def accumulated_mass_new(pos,mass,n,limits):
    
    r = np.sqrt((pos**2).sum(axis=1))
    # histograma acumulado
    histo_acum = scipy.stats.cumfreq(r, n, limits, weights=mass)
    # hallamos los radios manualmente
    positions = histo_acum[1]+ np.arange(1,n+1) * histo_acum[2]
  
    return positions, histo_acum[0]

#### Relacion densidad vs. radio

In [5]:
# Funcion de densidad
def density_dm_new(x,y):
    x,y = accumulated_mass_new(data_nuevo_ruta.dm['pos'],data_nuevo_ruta.dm['mass'],n=int(limites_materia_oscura[gal][1]*4),limits=limites_materia_oscura[gal])
    # np.roll re acomoda los datos desplazandolos una posicion
    x2 = np.roll(x,1)
    x2[0]=0
    # promedio del radio
    bins_middle = (x + x2 )/2
    
    x_cubico = x**3
    x_cubico_2 = np.roll(x_cubico,1)
    x_cubico_2[0]=0
    
    y2 = np.roll(y,1)
    y2[0]=0
    shell_mass = y - y2
    
    # \rho = (4/3)*pi*R^3
    dens = 3* shell_mass / ((4*np.pi)* (x_cubico - x_cubico_2) ) 

    return bins_middle, shell_mass, dens

#### Funciones para las graficas

In [6]:
# Plots de masa acumulada.
def plot_mass_new(bins,mass,filename):
    fig = plt.figure(figsize=(10, 7))

    plt.plot(bins,mass,'-k')

    plt.title('Masa acumulada de la Materia Oscura', fontsize= 18)
    plt.xlabel('Radio [Kpc]',fontsize=20)
    plt.ylabel('Masa Acumulada (Msol)')
    plt.grid()
    plt.savefig(filename)
    plt.clf()
    plt.close()

In [9]:
# Plot para la densidad.
def plot_density_new(bins,dens,filename):
    
    fig = plt.figure(figsize=(10, 7))
    
    plt.plot(bins, np.log10(dens), '-k')
   
    plt.title('Perfil de densidad de materia oscura', fontsize = 18)
    plt.xlabel('Radio [Kpc]', fontsize = 20)
    plt.ylabel(r'$\rho$ [$1x10^{10}$ Msol/$kpc^{2}$]', fontsize = 20)
    plt.grid()
    plt.savefig(filename)
    plt.clf()
    plt.close()

#### Generacion de graficas

In [10]:
# Generacion de plots

minsnap = 100
maxsnap = 700
stepsnap = 100

# definimos la fuente de los datos, para esto usamos los diccionarios que definimos mas arriba
for gal in ['G0','G1','G2','G3']:
    for snap in np.arange(minsnap,maxsnap,stepsnap): 
        snap = str(snap).zfill(3)
        # Aca definimos la ruta en la cual estan almacenados los snapshots
        data_nuevo_ruta = pynbody.load(data_nuevo+'/'+gal+'/snapshot_'+snap+'.hdf5')
        
        # Aca vamos a generar cada uno de los plots, para lo cual primero debemos definir
        # cada una de las funciones de masa acumulada y densidad.
        bins, mass = accumulated_mass_new(data_nuevo_ruta.dm['pos'],data_nuevo_ruta.dm['mass'],n=int(limites_materia_oscura[gal][1]*4),limits=limites_materia_oscura[gal])
        
        bins_middle, shell_mass, dens = density_dm_new(bins, mass)
       
        plot_mass_new(bins,mass,filename='Materia_oscura/Materia_Oscura_Acumulada_'+gal+'_dm_'+snap+'.jpg')
       
        plot_density_new(bins_middle,dens,filename='Materia_oscura/Materia_Oscura_Densidad_'+gal+'_dm_'+snap+'.jpg')

/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No unit information found: using gadget-units.
  warnings.warn(
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No unit information found: using gadget-units.
  warnings.warn(
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No un

/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No unit information found: using gadget-units.
  warnings.warn(
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No unit information found: using gadget-units.
  warnings.warn(
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No un